# Agent-based bacteria colony simulation

In [105]:
# To - do

# multiplication of bacteria
# eating food by bacteria
# chart with bacteria growth with time
# random moving of bacteria

In [106]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import itertools
import random
from IPython.display import HTML

In [107]:

class Bacterium:
    def __init__(self):  # 1 - young, 2 - ready to divide, 3 - dead
        
        self.state = 1
        self.hunger_level = 0 # if hunger_level == 10 -> dies
        self.can_use_galactose = 0
        self.maturity = 0
        
    def get_hungry(self, speed=1):
        self.hunger_level += 1*speed
    
    def learn_to_use_galactose(self):
        if self.hunger_level > 6:
            self.can_use_galactose = 1
            
    def mature(self):
        if self.hunger_level < 5:
            self.maturity +=1
            
    def get_ready_to_divide(self, is_bacteriostatic):
        if self.maturity > 4 and is_bacteriostatic == 0:
            self.state = 2
            
    def after_division(self): # we use it for 2 cells that are created after division
        self.state = 1
        self.hunger_level = 2
        self.can_use_galactose = 0
        self.maturity = 0
    
    def die(self, is_antibiotic):
        if is_antibiotic==1 or self.hunger == 10:
            self.state = 3 # bacteria is dead
    
    def getColour(self):
        if self.state == 1:
            return 255
        elif self.state == 2:
            return 200
        else:
            return 150  # for dead cells
        

In [108]:
class Food:
    def __init__(self, FoodType):
        self.FoodType = FoodType   # 1 - glucose, 2 - galactose, 0 - no food
        
    def getColour(self):
        if self.FoodType == 0:   # no food
            return 0
        elif self.FoodType == 1:   # glucose - light blue
            return 100
        elif self.FoodType == 2:   #galactose - dark blue
            return 50
        
    def __repr__(self):
        return str(self.FoodType)
        

In [109]:
class Board:
    
    def __init__(self, xSize, ySize, food_items=[]):
        self.food_items = []
        
        
        
        for i in range(0, xSize):
            row = []
            for j in range(0, ySize):
                row.append(Food(0))
            self.food_items.append(row)
        
        
        
        # initialize the board with a few food items (glucose - 1 and galactose - 2)
        
        self.food_items[1][3] = Food(1)
        self.food_items[3][7] = Food(2)
        
        
        #we only initialize bacteria where there is no food - Food with 0
        self.bacteria = []
        
        
        #create list of coordinates that are available for bacteria
        
        available_for_bacteria = []
        
        for i in range(0, xSize):
            for j in range(0, ySize):
                if self.food_items[i][j].FoodType == 0:
                    available_for_bacteria.append([i,j])
                    
        bacteria_location1 = available_for_bacteria[1]
        bacteria_location2 = available_for_bacteria[5]
        
        
        # create bacteria on empty spaces
        
        self.food_items[bacteria_location1[0]][bacteria_location1[1]] = Bacterium()
        self.food_items[bacteria_location2[0]][bacteria_location2[1]] = Bacterium()
        
                
    def printBoard(self):
        print('printing the board')
        print(self.food_items)
                                                    
    def randomBoard(self, xSize, ySize):  #TODO - initializing a random board with food
        random_board = Board(xSize, ySize)
        return

        
    def iteration(self): 
        #print('iteration!')
        self.printBoard()
        """
        self.calculateField()
        
        for i in range(1,len(self.food_items)-1):
            for j in range(1, len(self.food_items[0])-1):
                self.food_items[i][j].isBlocked = False
                
        
        for i in range(1,len(self.food_items)-1):
            for j in range(1, len(self.food_items[0])-1):
                self.food_items[i][j].move()
        """
    
    def clear(self):
        """
        for i in range(0,len(self.food_items)):
            for j in range(0, len(self.food_items[0])):
                self.food_items[i][j].clear_static()
        #print('robie clear booard')
        self.calculateField()
        """
    
    def redraw(self):
        # Returns numpy array of colours
        print('redraws!')
        fig = np.zeros((len(self.food_items),len(self.food_items[0])))
        for i in range(0,len(self.food_items)):
            for j in range(0, len(self.food_items[0])):
                fig[i][j] = self.food_items[i][j].getColour()
        print('fig:', fig)
        return fig
    
    
                
            

In [110]:
def update(frameNum, board, im, xSize, ySize):
    if(frameNum == 0):
        board.randomBoard(xSize, ySize)
        
        im.set_data(board.redraw()) 
    else:   
        board.iteration()
        #print('field map: ', board.getFieldMap())
        im.set_data(board.redraw()) 
    return im,

In [111]:
%%capture
# Parameters
steps = 100
# Config
fig, ax = plt.subplots(figsize=(10,10))
plt.axis('off')


In [112]:
#Run Simulation
import random 

xSize = 10
ySize= 10
board = Board(xSize,ySize,[])

board.clear()

im = ax.imshow(board.redraw(),  vmin=0, vmax=255)
ani = animation.FuncAnimation(fig, update, frames = steps, fargs = [board, im, xSize, ySize])
HTML(ani.to_jshtml())


redraws!
fig: [[  0. 255.   0.   0.   0. 255.   0.   0.   0.   0.]
 [  0.   0.   0. 100.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.  50.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
redraws!
fig: [[  0. 255.   0.   0.   0. 255.   0.   0.   0.   0.]
 [  0.   0.   0. 100.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.  50.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0. 